# LSTM+概率模型
## 1 思路来源
LSTM+概率模型思路来源于文章[An Integrated Missing-Data Tolerant Model for Probabilistic PV Power Generation Forecasting](https://ieeexplore.ieee.org/abstract/document/9697353)。
>在所提出的方法中，设计了一个具有<b>概率输出层的自回归LSTM网络</b>来模拟光伏发电配置文件中的时间依赖性和固有不确定性。负对数似然（而不是 MSE）用于指导模型训练。通过这种方式，<b>传统的基于LSTM的点预测以分布参数的形式扩展到概率预测</b>。基于这样的网络架构，一个模型可以同时高效地完成插补和概率预测任务。
## 2 网络架构
### 2.1 整体架构
![img1](https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/59/9927170/9697353/xu1-3146982-small.gif)
最终输出结果的概率分布是：${\rm{P}}({x_{{t_0} + 1:T}}{\rm{|}}{x_{{\rm{1}}:{t_0}}}){\rm{ = }}\prod\limits_{{\rm{t = }}{{\rm{t}}_0}}^T {{P_{\rm{l}}}} \;(\;{x_{t + 1}}{\rm{|}}{\theta _{\rm{t}}})$，也就是曲线的条件分布是所有散点概率分布的乘积。
### 2.2 LSTM
略
### 2.3 概率模型
其中概率层的计算公式为：${\theta _{\rm{t}}} = \phi \;({{\bf{h}}_{\rm{t}}},{{\bf{\Theta }}_{\rm{p}}})$，$\phi$是概率层函数，${{\bf{h}}_{\rm{t}}}$是隐藏层输出，${{\bf{\Theta }}_{\rm{p}}}$是概率层函数。

高斯分布是最常用的分布，因此概率层设置参数为：${\rm{\theta \ = \ (\mu,\ \sigma)}}$，给出均值与标准差。

使用两个全连接层实现与LSTM隐藏层输出的连接，架构如图：![img2](https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/59/9927170/9697353/xu2-3146982-small.gif)

由此可计算概率层参数：
\
${\mu _{\rm{t}}} = {\bf{w}}_\mu ^{\rm{T}}\;{{\bf{h}}_{\rm{t}}} + {{\rm{b}}_\mu }$，由全连接层给出；
\
${\sigma _{\rm{t}}} = softplus({\bf{w}}_\sigma ^{\rm{T}}\;{{\bf{h}}_{\rm{t}}} + {{\rm{b}}_\sigma }) = {\rm{log}}({\rm{1 + exp}}({\bf{w}}_\sigma ^{\rm{T}}\;{{\bf{h}}_{\rm{t}}} + {{\rm{b}}_\sigma }))$，由全连接层通过softplus函数给出，softplus函数的作用是保持输出在$(0, + \infty )$有良好的非线性建模能力。
\
代入高斯分布即可计算每一个点的概率分布：${P_{\rm{l}}}({x_{t + 1}}{\rm{|}}\;{\theta _{\rm{t}}})\;\;{\rm{ = }}\;{P_{\rm{l}}}\;({x_{t + 1}}{\rm{|}}\;{\mu _{\rm{t}}},{\sigma _{\rm{t}}}){\rm{ = }}{({\rm{2}}\pi {\sigma _{\rm{t}}}^2)^{{\rm{ - 1/2}}}}\;{{\rm{e}}^{{\rm{ - }}\frac{{{{({x_{t + 1}}{\rm{  -  }}{\mu _t})}^2}}}{{({\rm{2}}{\sigma _t}^2)}}}}$
## 3 训练
因为加入了概率层，因此不能使用MSE作为LOSS函数，选取负对数似然函数可以一次性完成神经网络所有层的优化（LSTM+概率层）。（为什么可以？）

设数据集为N个T步的数据集合：${\{ {x_{i,\;1:T}}\} _{{\rm{i}}\;{\rm{ = }}\;{\rm{1}},\; \ldots ,{\rm{N}}}}$
\
则有LOSS函数：${\cal L} = {\rm{ - }}\sum\limits_{{\rm{i}}\;{\rm{ = }}\;{\rm{1}}}^{\rm{N}} {\sum\limits_{{\rm{t}}\;{\rm{ = }}\;{\rm{0}}}^{\rm{T}} {{\rm{log}}} } \;{P_{\rm{l}}}\;(\;{x_{t + 1}}\;{\rm{|}}\;{\theta _{\rm{t}}})$，使用梯度下降法即可完成两层参数优化。